In [ ]:
import billboard
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import base64
from requests import post, get
import json
import time
from datetime import datetime, timedelta


In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
data_df = pd.read_csv("data_df.csv")

# Display sample data
data_df.head()

In [ ]:
data_df["Year"] = ""
data_df.head()

In [ ]:
# Convert 'Year' column to numeric values, coercing errors to NaN
data_df['Year'] = pd.to_numeric(data_df['Year'], errors='coerce')

# Fill NaN values (if any) with 0 (or any other placeholder you want)
data_df['Year'] = data_df['Year'].fillna(0)

# Convert 'Year' back to integers after filling NaN
data_df['Year'] = data_df['Year'].astype(int)

# Add 2006 to the 'Year' column for the first 100 rows
data_df.loc[:99, 'Year'] = 2006

# Add 1 to the 'Year' for every subsequent block of 100 rows (e.g., 2007, 2008, etc.)
for i in range(1, (len(data_df) // 100) + 1):
    start_idx = i * 100
    end_idx = (i + 1) * 100
    data_df.loc[start_idx:end_idx - 1, 'Year'] = 2006 + i

# Output the result
print(data_df[['Song Name', 'Year']].head(110))  # Display first 110 rows to check

In [ ]:
data_df.head(1900)

In [ ]:
data_df.drop('Release Date', axis=1, inplace=True)
data_df.head()

In [ ]:
data_df['Rank'] = (data_df.index % 100) + 1
data_df.head(101)

In [ ]:
data_df.rename(columns={'Song Name': 'Title'}, inplace=True)

In [ ]:
new_df = data_df[['Year', 'Rank', 'Title', 'Artist', 'Spotify ID', 'Genres', 'Duration', 'Popularity']]
new_df.head(202)

In [ ]:
# Split 'Artist' column into multiple columns
artist_split = new_df['Artist'].str.split(', ', expand=True)

# Rename new columns as 'feat1', 'feat2', etc.
artist_split.columns = [f'feat{i+1}' for i in range(artist_split.shape[1])]

# Merge the new columns into the DataFrame
update_df = pd.concat([new_df, artist_split], axis=1)

update_df.head()

In [ ]:
artist_drop_df = update_df.drop(columns=['Artist'])
artist_drop_df.head()

In [ ]:
# Rename 'feat1' to 'Artist'
renamed_df = artist_drop_df.rename(columns={'feat1': 'Artist'})
renamed_df.head()

In [ ]:
# Rename 'feat1' to 'Artist'
renamed_df2 = renamed_df.rename(columns={'feat2': 'First Feature', 'feat3': 'Second Feature'})
renamed_df2.head(50)

In [ ]:
split_cleaned_df = renamed_df2[['Year', 'Rank', 'Title', 'Artist', 'First Feature', 'Second Feature', 'Genres', 'Duration', 'Popularity', 'Spotify ID']]
split_cleaned_df.head(202)

In [ ]:
# Export the DataFrame to a JSON file
split_cleaned_df.to_json('split_cleaned_df.json', orient='records', lines=True)

In [ ]:
# Export the DataFrame to a csv file
split_cleaned_df.to_csv('split_cleaned_df.csv', index=False)